# Introdução

No *site* da CFB
https://www.infraestruturameioambiente.sp.gov.br/cfb/contatos/

In [ ]:
import os
import re
import time
import random
import requests
import numpy as np
import pandas as pd
import geopandas as gpd

from bs4 import BeautifulSoup
from osgeo import gdal, osr
from tqdm.notebook import trange, tqdm

# Dados Tabulares

## Scrapy *Site*

In [ ]:
url = 'https://www.infraestruturameioambiente.sp.gov.br/cfb/contatos/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
tag = soup.find('div', {'id': 'content'}).text

In [ ]:
list_main = []
for i in str(tag).split('\n'):
    if i == '':
        pass
    else:
        list_main.append(i)

# Create Table
df = pd.DataFrame({'data': list_main})
print(df[0:100])

## Ajusta de Tabela

In [ ]:
# Define os marcos iniciais
df['interval'] = df.apply(lambda x: x.name if 'CTR' in x['data'] else np.nan, axis=1)

# Remove Espaços em Branco
df['data'] = df.apply(lambda x: np.nan if x['data'] == '\xa0' else x, axis=1)

# Preenche Coluna
df['interval']  = df['interval'].ffill()

# Elimina Linhas com NaN
df = df.dropna()

# Result
df.head(10)

In [ ]:
list_temps = []
list_for = list(set(df['interval']))
print(list_for)
for i in list_for:
    df_temp = df[df['interval'] == i]
    list_temp = list(df_temp['data'])    
    list_temps.append(list_temp)

In [ ]:
for i in list_temps:    
    if len(i)==7:
        i.insert(5, None)

In [ ]:
df = pd.DataFrame(
    list_temps,
    columns = [
        'id_ctr',
        'end',
        'bairro_cidade',
        'cep',
        'telefone_1',
        'telefone_2',
        'email',
        'diretor',
    ])

df.head()

In [ ]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')
df

In [ ]:
for i in df.columns:
    print(set(df[i]))

## Funções

In [ ]:
def rename_nome(x):
    x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '  '  : ' ',
        
        # Basics
        ' Com '  : ' com ',
        ' Sobre ': ' sobre ',
        ' Da '   : ' da ',
        ' De '   : ' de ',
        ' Do '   : ' do ',
        ' Das '  : ' das ',
        ' Dos '  : ' dos ',
        ' A '    : ' a ',
        ' As '   : ' as ',
        ' Ao '   : ' ao ',
        ' Aos '  : ' aos ',
        ' E '    : ' e ',
        ' O '    : ' o ',
        ' Os '   : ' os ',
        
        # Erros
        '1ºten'  : '1º Ten',
        
        # Abreviações
        'Subten ' : 'Subtenente ',
        'Sub Ten' : 'Subtenente ',
        'Cap '    : 'Capitão ',
        'Ten '    : 'Tenente ',
        'Maj '    : 'Major ',
        'Cel '    : 'Coronel ',        
        'Sgt '    : 'Sargento ',
        ' Pm '    : ' PM ',
        '–': '-',
        'Registro/Sp': 'Registro - Sp',
        'São Bernardo do Campo- Sp': 'São Bernardo do Campo - Sp',
        ' - Sp': '',
        
        'R.': 'Rua',
        'Av.': 'Avenida',
        'Avenida Rodrigues Alves - Quadra 38 - Nº 138': 'Avenida Rodrigues Alves, 138 - Quadra 38',
        '133, Sala 23': '133 - Sala 23',

        # Empty
        'None' : '',
        'none' : '',
    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

## Renomeando Campos em Colunas

In [ ]:
df['bairro_cidade'] = df['bairro_cidade'].astype(str).apply(lambda x: rename_nome(x))
df[['bairro','municipio_sede']] = df['bairro_cidade'].str.split('-', expand=True)

df['bairro'] = df['bairro'].astype(str).apply(lambda x: rename_nome(x))
df['municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['telefone_1'] = df['telefone_1'].astype(str).apply(lambda x: rename_nome(x))
df['telefone_2'] = df['telefone_2'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['cep'] = df['cep'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: rename_nome(x))
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.rsplit('-', 1)[0])
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.upper())
df['id_ctr'] = df['id_ctr'].astype(str).apply(lambda x: x.strip().replace(' ', ''))

In [ ]:
df['end'] = df['end'].astype(str).apply(lambda x: rename_nome(x))
df['endereco'] = df['end'].astype(str).apply(lambda x: x.split(',', 1)[0])
df['numero'] = df['end'].astype(str).apply(lambda x: x.split(',', 1)[1])
df[['numero','complemento']] = df['numero'].str.split('-', n=1, expand=True)

df['numero'] = df['numero'].astype(str).apply(lambda x: rename_nome(x))
df['complemento'] = df['complemento'].astype(str).apply(lambda x: rename_nome(x))

df

## Reordena Colunas

In [ ]:
df = df[[
    'id_ctr',
    'endereco',
    'numero',
    'complemento',
    'cep',
    'bairro',
    'municipio_sede',
    'telefone_1',
    'telefone_2',
    'email',
    'diretor'
]]

df

## Salva

In [ ]:
# Results
df.to_csv(
    os.path.join('data', 'tabs', 'tab_ctr.csv'),
    index=False,
)
df